In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("dataset.csv")

# The data type of perpetrator's age was not seen as int initially, so there was the need to treat those values and 
# transform them to NaN (there is only one row with an NA value in 'Perpetrator Age')
df['Perpetrator Age'] = pd.to_numeric(df['Perpetrator Age'], errors='coerce').astype('Int64')
df.dropna(inplace=True)

C:\Users\ingri\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.dtypes

Record ID            int64
Agency Code         object
Agency Name         object
Agency Type         object
City                object
State               object
Year                 int64
Month               object
Incident             int64
Crime Type          object
Crime Solved        object
Victim Sex          object
Victim Age           int64
Victim Race         object
Perpetrator Sex     object
Perpetrator Age      Int64
Perpetrator Race    object
Relationship        object
Weapon              object
Record Source       object
dtype: object

In [4]:
#number of lines with NA values count
nan_counts = df.isna().sum()
nan_counts

Record ID           0
Agency Code         0
Agency Name         0
Agency Type         0
City                0
State               0
Year                0
Month               0
Incident            0
Crime Type          0
Crime Solved        0
Victim Sex          0
Victim Age          0
Victim Race         0
Perpetrator Sex     0
Perpetrator Age     0
Perpetrator Race    0
Relationship        0
Weapon              0
Record Source       0
dtype: int64

In [5]:
df

,Record ID,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type,Crime Solved,Victim Sex,Victim Age,Victim Race,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Relationship,Weapon,Record Source
0,1,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,January,1,Murder or Manslaughter,Yes,Male,14,Native American/Alaska Native,Male,15,Native American/Alaska Native,Acquaintance,Blunt Object,FBI
1,2,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,1,Murder or Manslaughter,Yes,Male,43,White,Male,42,White,Acquaintance,Strangulation,FBI
2,3,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,2,Murder or Manslaughter,No,Female,30,Native American/Alaska Native,Unknown,0,Unknown,Unknown,Unknown,FBI
3,4,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,1,Murder or Manslaughter,Yes,Male,43,White,Male,42,White,Acquaintance,Strangulation,FBI
4,5,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,2,Murder or Manslaughter,No,Female,30,Native American/Alaska Native,Unknown,0,Unknown,Unknown,Unknown,FBI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638449,638450,WY01500,Park County,Sheriff,Park,Wyoming,2014,January,1,Murder or Manslaughter,No,Male,30,White,Unknown,0,Unknown,Unknown,Handgun,FBI
638450,638451,WY01700,Sheridan County,Sheriff,Sheridan,Wyoming,2014,June,1,Murder or Manslaughter,Yes,Male,62,White,Male,57,White,Acquaintance,Handgun,FBI
638451,638452,WY01701,Sheridan,Municipal Police,Sheridan,Wyoming,2014,September,1,Murder or Manslaughter,Yes,Female,0,Asian/Pacific Islander,Female,22,Asian/Pacific Islander,Daughter,Suffocation,FBI
638452,638453,WY01800,Sublette County,Sheriff,Sublette,Wyoming,2014,December,1,Murder or Manslaughter,Yes,Male,55,White,Male,31,White,Stranger,Knife,FBI


## Tasks

a) Identify which variables are categorical, discrete and continuous in the chosen data set and show
using some visualization or plot. Explore whether there are missing values for any of the variables.

b) Calculate the statistical parameters (mean, median, minimum, maximum, and standard deviation)
for each of the numerical variables.

c) Apply Min-Max Normalization, Z-score Standardization and Robust scalar on the numerical data
variables.

d) Line, Scatter and Heatmaps can be used to show the correlation between the features of the
dataset.

e) Graphics and descriptive understanding should be provided along with Data Exploratory analysis
(EDA). Identify subgroups of features that can explore some interesting facts.

f) Apply dummy encoding to categorical variables (at least one variable used from the data set) and
discuss the benefits of dummy encoding to understand the categorical data.

g) Apply PCA with your chosen number of components. Write up a short profile of the first few
components extracted based on your understanding.

h) What is the purpose of dimensionality reduction? Explore the situations where you can gain the
benefit of dimensionality reduction for data analysis.

-----------------

a)

b) Numerical Values: Year, Victim Age, Perpetrator Age

## Mean Calculation

In [13]:
year_mean = df['Year'].mean()
year_mean = round(year_mean, 1)
victim_age_mean = df['Victim Age'].mean()
victim_age_mean = round(victim_age_mean, 2)
perpetrator_age_mean = df['Perpetrator Age'].mean()
perpetrator_age_mean = round(perpetrator_age_mean, 2)

## Mean Values

In [14]:
print("The mean for the Year values is: ", year_mean)
print("The mean for the Victim Age values is: ", victim_age_mean)
print("The mean for the Year values is: ", perpetrator_age_mean)

The mean for the Year values is:  1995.8
The mean for the Victim Age values is:  35.03
The mean for the Year values is:  20.32


## Median calculation

In [19]:
year_median = df['Year'].median()
victim_age_median = df['Victim Age'].median()
perpetrator_age_median = df['Perpetrator Age'].median()

In [20]:
print("The median for the Year values is: ", year_median)
print("The median for the Victim Age values is: ", victim_age_median)
print("The median for the Year values is: ", perpetrator_age_median)

The median for the Year values is:  1995.0
The median for the Victim Age values is:  30.0
The median for the Year values is:  21.0
